#### libraries

In [0]:
# sharepoint modules
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.files.file import File

# local files handling when reading input
import os
# in-memory handling for writing files for export
# from io import BytesIO

# to read/write CSV in sharepoint
import pandas as pd
'''
If using spark.read.csv() -> AnalysisException: Incompatible format detected.
Delta format not working either.
Eg:
  dfr = spark.read \
      .format("csv") \
      .option("header", "true") \
      .load(download_path)
'''

from pyspark.sql import functions as F

#### `general settings` to work with sharepoint

In [0]:
# sharepoint working site
site_url = "https://test-company.sharepoint.com/sites/SASIPAutomationProject/"
# init client context with credentials
client = ClientContext(site_url).with_credentials(UserCredential("admin-user@test-company.com", dbutils.secrets.get('secret-scope-name', 'secret-key')))

# fold_link = "https://test-company.sharepoint.com/:f:/s/SASIPAutomationProject/EoAbcsDcoSJCpvyTPSRBzscBwYb-lowtdDwUKh7kdTfHDg?e=l13thZ"

# fold_obj = client.web.get_folder_by_guest_url_extended(fold_link).execute_query()
# sharepoint_files = fold_obj.files   
# client.load(sharepoint_files).execute_query()

#### create individual `dataframes from CSV files` (comma separated)

In [0]:
input_files = {
  "exclusion__df" : "https://test-company.sharepoint.com/:f:/s/SASIPAutomationProject/ErUH64YzYENHtWrA02EEDV0BH9SUUQ1p5i1qlkUPnPkWLg?e=20GjiH",
  "role_map__df" : "https://test-company.sharepoint.com/:f:/s/SASIPAutomationProject/EknYQFKVeTdDnEEvysgkUKAB2QnZdAM8kpHsDxJxYbpUVw?e=lY8zSW",
  "audit_detail_mtd__df": "https://test-company.sharepoint.com/:f:/s/SASIPAutomationProject/Eq2mqMvKe1FIlVr5aKvM00oBo_1lehXBue_vGL9tW-ciTA?e=sx3wdh",
  "account_list__df" : "https://test-company.sharepoint.com/:f:/s/SASIPAutomationProject/EnzRQOSFaSdOrcJaYXbxGTkBdzcvIXaK9WIrjAQdgKvRqg?e=GZne0M"
}

In [0]:
for key, value in input_files.items():
  
  print(f'creating dataframe: {key}')
  fold_link = value

  fold_obj = client.web.get_folder_by_guest_url_extended(fold_link).execute_query()
  sharepoint_files = fold_obj.files   
  # client to work with sharepoint_files
  client.load(sharepoint_files).execute_query()
  
  try:
    working_file = sharepoint_files[0]

    download_path = os.path.join(os.getcwd(), str(working_file))
    # apply sharepoint library method
    file_response = File.read(working_file)
    # Save file locally for pandas to read next
    with open(download_path, 'wb') as output_file:
      output_file.write(file_response)  
    
    # read file with pandas and convert to spark dataframe.
    # "low_memory=False" avoids data type error, alternatively use a dictionary and the dtype option to define data types.
    # elimintate empty rows if any.
    df = pd.read_csv(download_path, low_memory=False).dropna(how='all') 
    globals()[key] = spark.createDataFrame(df)

    # delete unnecesary elements
    os.remove(download_path)
    del df

  except:
    print(f"no file for {key}")

creating dataframe: exclusion__df
creating dataframe: role_map__df
creating dataframe: audit_detail_mtd__df
creating dataframe: account_list__df

In [0]:
exclusion__df.show(10, truncate=False)
# display(exclusion__df.limit(10))

+------------------------------------+------------------+------------+------------+-----------------+-------------------+---------------------+-------+
Account Name |Account Id |Retail Store|KAM |Reason |Account Owner |Account Owner Manager|Exclude|
+------------------------------------+------------------+------------+------------+-----------------+-------------------+---------------------+-------+
Captain Liquor Distribution |aCaD0000000PFBLKA4|No |Chad Haverly|No Retail element|Cecil Bagley |Hein Stander |1.0 |
Distri-Liq (Pretoria) |aCaD0000000PF0tKAG|No |Chad Haverly|No Retail element|Monique Bester |Charles Maseko |1.0 |
Distri-Liq (Rustenburg) |aCaD0000000PFNcKAO|No |Chad Haverly|No Retail element|Kagiso Motau |Given Mmekoa |1.0 |
Durban North Liquors (Phoenix) (Ind)|aCaD0000000gbxoKAA|No |Chad Haverly|No Retail element|Mphokuhle Cele |Antara Ramouthar |1.0 |
Norman Goodfellows (Paarden Eiland) |aCaD0000000PG0WKAW|No |Chad Haverly|No Retail element|Chris Bodenham |Robbie Van Eck |1.0 |
Norman Goodfellows (Wynberg) |aCaD0000000fziVKAQ|No |Chad Haverly|No Retail element|Pravin Hardin |Hein Stander |1.0 |
Panjivan Liquors |aCaD0000000PF2RKAW|No |Chad Haverly|No Retail element|Sibaphiwe Sali |Brandon Pillay |1.0 |
Panjivan Pietermaritzburg |aCaD0000000PFYgKAO|No |Chad Haverly|No Retail element|Noluthando Maselana|Andrew Morris |1.0 |
Panjivan Swelani Liquors |aCaD0000000PF03KAG|No |Chad Haverly|No Retail element|Kuben Pillay |Andrew Morris |1.0 |
Panjivans Big Ben Liquor Wholesalers|aCaD0000000PFM6KAO|No |Chad Haverly|No Retail element|Sibaphiwe Sali |Brandon Pillay |1.0 |
+------------------------------------+------------------+------------+------------+-----------------+-------------------+---------------------+-------+
only showing top 10 rows

In [0]:
# display(role_map__df.limit(10))
role_map__df.show(10, truncate=False)

+-------------------+-----------------------------------+-----------------------+
Area Sales Director|Account Owner Manager/Sales Manager|Account Owner/Sales Rep|
+-------------------+-----------------------------------+-----------------------+
Celeste Carstens |Andrew Morris |Chimone Horst |
Celeste Carstens |Andrew Morris |Kuben Pillay |
Celeste Carstens |Andrew Morris |Lebogang Maaga |
Celeste Carstens |Andrew Morris |Mphokuhle Cele |
Celeste Carstens |Brandon Pillay |Vincent Msowoya |
Celeste Carstens |Brandon Pillay |Qhamani Bam |
Celeste Carstens |Brandon Pillay |Siphesihle Magwaza |
Celeste Carstens |Brandon Pillay |Themba Myeni |
Celeste Carstens |Brandon Pillay |Tsuso Tserere |
Celeste Carstens |Brandon Pillay |Pilani Nondabula |
+-------------------+-----------------------------------+-----------------------+
only showing top 10 rows

In [0]:
# display(audit_detail_mtd__df.limit(10))
audit_detail_mtd__df.show(10, truncate=False)

+-----------------+---------+--------------------------------------+------------------+---------------------+---------------------------------+-------------------------------------------+---------+------+----------+----------+-------------+---------------+----------------------+-------------+---------------+----------------+---------+
Audit Datetime |Channel |Account Name |Account 18 ID |Created By: Full Name|Brand-Quality: Brand Quality Name|Brand-Quality-Size: Brand Quality Size |Category |Listed|Visibility|Activation|House Pouring|Number of Faces|Regular Price Currency|Regular Price|Store Audit ID |Store Audit Item|Available|
+-----------------+---------+--------------------------------------+------------------+---------------------+---------------------------------+-------------------------------------------+---------+------+----------+----------+-------------+---------------+----------------------+-------------+---------------+----------------+---------+
2024/10/10, 07:51|OFF TRADE|Thandile Bottle Store (EFT OD) |001Tn000003F9n9IAC|Zwai Sonjica |null |Luc Belaire Rare Rose 750ml |CHAMPAGNE|1 |0 |0 |0 |1.0 |ZAR |549.0 |a2yTn000000jpHh|SAI-0075621124 |1 |
2024/10/08, 13:41|OFF TRADE|Tops Mdantsane City |001Tn0000066FIfIAM|Zwai Sonjica |null |Luc Belaire Luxe Rose Fantome 750ml |CHAMPAGNE|0 |0 |0 |0 |null |null |null |a2yTn000000jQ3G|SAI-0075257574 |0 |
2024/10/08, 11:51|OFF TRADE|Checkers L/Shop Zevenwacht (39061) |001Tn000003FIswIAG|Curven Swiegelaar |null |Luc Belaire Bleu 750ml |CHAMPAGNE|0 |0 |0 |0 |null |null |null |a2yTn000000jS1q|SAI-0075285746 |0 |
2024/10/14, 12:29|OFF TRADE|Agrimark Riebeek West (1907)(OK) |001Tn000003FIHOIA4|Fabian Jacobs |null |Ballantines 7YO 6x750ml |WHISKY |0 |0 |0 |0 |null |null |null |a2yTn000000kFnG|SAI-0075741899 |0 |
2024/10/16, 14:11|OFF TRADE|Tops Stone Square 35707 |001Tn000003FGlcIAG|Curven Swiegelaar |null |Absolut Cocktail Berry Vodkarita 4x300ml |RTD |1 |0 |0 |0 |1.0 |ZAR |134.99 |a2yTn000000kYrX|SAI-0075898351 |1 |
2024/10/17, 14:42|OFF TRADE|PNP Corporate - Montana (NC79) |001Tn000003FDabIAG|Mdu Shongwe |null |Malibu Cocktail Strawberry Daiquiri 4x300ml|RTD |1 |0 |0 |0 |null |ZAR |119.99 |a2yTn000000kiyv|SAI-0075966207 |1 |
2024/10/01, 12:53|OFF TRADE|PNP Family - King Williams Town (EF16)|001Tn000003F9ubIAC|Zwai Sonjica |null |Jameson Caskmates Stout 12x750ml |WHISKY |1 |0 |0 |0 |3.0 |ZAR |449.0 |a2yTn000000iYMB|SAI-0074793416 |1 |
2024/10/04, 14:57|OFF TRADE|Checkers L/Shop Castle Rock (32899) |001Tn000003FGy3IAG|Themba Myeni |null |Chivas XV 750 |WHISKY |1 |0 |0 |0 |2.0 |ZAR |739.99 |a2yTn000000j5gL|SAI-0075100043 |1 |
2024/10/02, 10:12|OFF TRADE|Tops Spargs Idutywa 46081 |001Tn000003FEvSIAW|Mlungisi Magwaxaza |null |Beefeater Blood Orange 750ml |GIN |1 |0 |0 |0 |1.0 |ZAR |359.0 |a2yTn000000ipgN|SAI-0074950426 |1 |
2024/10/21, 11:14|OFF TRADE|Shoprite L/Shop Porterville (92643) |001Tn000003FAuBIAW|Tyrone Engelbrecht |null |Inverroche Amber 750ml |GIN |0 |0 |0 |0 |null |null |null |a2yTn000000l5h3|SAI-0076149544 |0 |
+-----------------+---------+--------------------------------------+------------------+---------------------+---------------------------------+-------------------------------------------+---------+------+----------+----------+-------------+---------------+----------------------+-------------+---------------+----------------+---------+
only showing top 10 rows

In [0]:
# display(account_list__df.limit(10))
account_list__df.show(10, truncate=False)

+---------+-------------+------------------+---------------------------------------+------------------+----------------------------------------------------------------------------------+-------------+---------------------+-------------------------------+-------------------------+--------------------+-----------------+---------------------+-------------------------+----------------------------+----------------------------------+-----------------------------+------------------------------+------+-------------------------------------+-------------+-----------------+------------+----------------------------------+------+-------------+------------------------------+
Channel |Business Type|Customer No# (ERP)|Account Name |Account 18 ID |Street |City |Outlet Classification|PRS Group Outlet Type (EU) name|PRS Outlet Type (EU) name|PRS Image Level Name|Chain |Taxonomy Level 1 |PRS Volume Potential Name|PRS Business Potential: Name|PRS Brand Building Potential: Name|Outlet Geolocation (Latitude)|Outlet Geolocation (Longitude)|Status|Visit Frequency: Visit Frequency Name|Preferred Day|Owner: Full Name |Country |Territory: Territory Name |Region|Company Type |Key Account Manager: Full Name|
+---------+-------------+------------------+---------------------------------------+------------------+----------------------------------------------------------------------------------+-------------+---------------------+-------------------------------+-------------------------+--------------------+-----------------+---------------------+-------------------------+----------------------------+----------------------------------+-----------------------------+------------------------------+------+-------------------------------------+-------------+-----------------+------------+----------------------------------+------+-------------+------------------------------+
OFF TRADE|Outlet |37202 |Game Liquors (Zevenwacht) 327 |001Tn000003FFmpIAG|Shop 220, Zevenwacht Mall, Van Riebeeck Rd, Kuilsriver |Bellville |Retail Franchise |ZA RETAIL GROCERY |Group Corporate |null |Game Liquors |Game Liquors |null |High |High |-34,000,706,872,934,400 |18,726,913,360,896,200 |Active|2/Month W1 & 3 |Thursday |Curven Swiegelaar|South Africa|Cape Hybrid Inland (1) - Off Trade|South |Sales Service|null |
OFF TRADE|Outlet |52260 |Liquor City (Somerset West) (EFT 24hrs)|001Tn000003FJEkIAO|Sh 401 Somerset Value Mart |Somerset West|Retail Franchise |ZA RETAIL OTHER |Group Corporate |null |Liquor City |Liquor City Corporate|null |High |High |-34,085,311,600,000,000 |18,824,000,400,000,000 |Active|Weekly |Tuesday |Curven Swiegelaar|South Africa|Cape Hybrid Inland (1) - Off Trade|South |Sales Service|Daryl Pillay |
OFF TRADE|Outlet |63685 |Tops Haasendal 36118 |001Tn000003FKeiIAG|Sh 47 3 Haasendal Boulevard |Kuils River |Retail Grocery |ZA RETAIL GROCERY |Group Franchise |null |Spar Group |Spar WC |null |High |High |-33,895,769,807,723,400 |18,502,715,078,192,700 |Active|Weekly |Tuesday |Curven Swiegelaar|South Africa|Cape Hybrid Inland (1) - Off Trade|South |Sales Service|Jacques Weideman |
OFF TRADE|Outlet |null |Woolworths Waterstone Village |001Tn0000066FIjIAM|Shop 38/39 Waterstone Village Corner of R44& main Rd,Somerset West, Cape Town,7130|Somerset West|null |ZA RETAIL GROCERY |Group Corporate |null |Woolworths |null |null |High |High |-34,070,497,300,000,000 |18,827,970,800,000,000 |Active|Weekly |Wednesday |Curven Swiegelaar|South Africa|Cape Hybrid Inland (1) - Off Trade|South |Sales Service|Hein Stander |
OFF TRADE|Outlet |null |Checkers Kenilworth |001Tn000003FDIrIAO|kenilworth Centre , shop 17 , Chichester road , claremont |Kenilworth |null |ZA RETAIL GROCERY |Group Corporate |null |Shoprite Checkers|null |null |High |High |-33,988,986,800,000,000 |18,485,150,000,000,000 |Active|2/Month W1 & 4 |Monday |Sibongile Magazi |South Africa|Cape Hybrid Inland (2) - Off Trade|South |Sales Service|Hein Stander |
OFF TRADE|Outlet |11878 |Makro (Ottery) |001Tn000003FDSkIAO|cnr Ottery/Ol

#### original tables `Transformation`

In [0]:
# Extract the date as a string and create a new column 'Date'
# Add a new column 'Audit' with a value of 1 for all rows (all of those are audited, boolean)
audit_detail_mtd__df = audit_detail_mtd__df \
  .withColumn( 
    "Date", audit_detail_mtd__df["Audit Datetime"].substr(1, 10)
  ) \
  .withColumn(   
     "Audit Complete", F.lit(1)
  )
# display(audit_detail_mtd__df.limit(10))
audit_detail_mtd__df.show(10, truncate=False)

+-----------------+---------+--------------------------------------+------------------+---------------------+---------------------------------+-------------------------------------------+---------+------+----------+----------+-------------+---------------+----------------------+-------------+---------------+----------------+---------+----------+--------------+
Audit Datetime |Channel |Account Name |Account 18 ID |Created By: Full Name|Brand-Quality: Brand Quality Name|Brand-Quality-Size: Brand Quality Size |Category |Listed|Visibility|Activation|House Pouring|Number of Faces|Regular Price Currency|Regular Price|Store Audit ID |Store Audit Item|Available|Date |Audit Complete|
+-----------------+---------+--------------------------------------+------------------+---------------------+---------------------------------+-------------------------------------------+---------+------+----------+----------+-------------+---------------+----------------------+-------------+---------------+----------------+---------+----------+--------------+
2024/10/10, 07:51|OFF TRADE|Thandile Bottle Store (EFT OD) |001Tn000003F9n9IAC|Zwai Sonjica |null |Luc Belaire Rare Rose 750ml |CHAMPAGNE|1 |0 |0 |0 |1.0 |ZAR |549.0 |a2yTn000000jpHh|SAI-0075621124 |1 |2024/10/10|1 |
2024/10/08, 13:41|OFF TRADE|Tops Mdantsane City |001Tn0000066FIfIAM|Zwai Sonjica |null |Luc Belaire Luxe Rose Fantome 750ml |CHAMPAGNE|0 |0 |0 |0 |null |null |null |a2yTn000000jQ3G|SAI-0075257574 |0 |2024/10/08|1 |
2024/10/08, 11:51|OFF TRADE|Checkers L/Shop Zevenwacht (39061) |001Tn000003FIswIAG|Curven Swiegelaar |null |Luc Belaire Bleu 750ml |CHAMPAGNE|0 |0 |0 |0 |null |null |null |a2yTn000000jS1q|SAI-0075285746 |0 |2024/10/08|1 |
2024/10/14, 12:29|OFF TRADE|Agrimark Riebeek West (1907)(OK) |001Tn000003FIHOIA4|Fabian Jacobs |null |Ballantines 7YO 6x750ml |WHISKY |0 |0 |0 |0 |null |null |null |a2yTn000000kFnG|SAI-0075741899 |0 |2024/10/14|1 |
2024/10/16, 14:11|OFF TRADE|Tops Stone Square 35707 |001Tn000003FGlcIAG|Curven Swiegelaar |null |Absolut Cocktail Berry Vodkarita 4x300ml |RTD |1 |0 |0 |0 |1.0 |ZAR |134.99 |a2yTn000000kYrX|SAI-0075898351 |1 |2024/10/16|1 |
2024/10/17, 14:42|OFF TRADE|PNP Corporate - Montana (NC79) |001Tn000003FDabIAG|Mdu Shongwe |null |Malibu Cocktail Strawberry Daiquiri 4x300ml|RTD |1 |0 |0 |0 |null |ZAR |119.99 |a2yTn000000kiyv|SAI-0075966207 |1 |2024/10/17|1 |
2024/10/01, 12:53|OFF TRADE|PNP Family - King Williams Town (EF16)|001Tn000003F9ubIAC|Zwai Sonjica |null |Jameson Caskmates Stout 12x750ml |WHISKY |1 |0 |0 |0 |3.0 |ZAR |449.0 |a2yTn000000iYMB|SAI-0074793416 |1 |2024/10/01|1 |
2024/10/04, 14:57|OFF TRADE|Checkers L/Shop Castle Rock (32899) |001Tn000003FGy3IAG|Themba Myeni |null |Chivas XV 750 |WHISKY |1 |0 |0 |0 |2.0 |ZAR |739.99 |a2yTn000000j5gL|SAI-0075100043 |1 |2024/10/04|1 |
2024/10/02, 10:12|OFF TRADE|Tops Spargs Idutywa 46081 |001Tn000003FEvSIAW|Mlungisi Magwaxaza |null |Beefeater Blood Orange 750ml |GIN |1 |0 |0 |0 |1.0 |ZAR |359.0 |a2yTn000000ipgN|SAI-0074950426 |1 |2024/10/02|1 |
2024/10/21, 11:14|OFF TRADE|Shoprite L/Shop Porterville (92643) |001Tn000003FAuBIAW|Tyrone Engelbrecht |null |Inverroche Amber 750ml |GIN |0 |0 |0 |0 |null |null |null |a2yTn000000l5h3|SAI-0076149544 |0 |2024/10/21|1 |
+-----------------+---------+--------------------------------------+------------------+---------------------+---------------------------------+-------------------------------------------+---------+------+----------+----------+-------------+---------------+----------------------+-------------+---------------+----------------+---------+----------+--------------+
only showing top 10 rows

In [0]:
account_list__df.createOrReplaceTempView("account_list__view")
role_map__df.createOrReplaceTempView("role_map__df__view")
audit_detail_mtd__df.createOrReplaceTempView("audit_detail_mtd__view")
exclusion__df.createOrReplaceTempView("exclusion__view")

In [0]:
account_list_final__df = \
  spark.sql('''
    SELECT 
      ac.*,
      ro.`Area Sales Director` ,
      ro.`Account Owner Manager/Sales Manager` AS `Account Owner Manager`,
      CASE
        WHEN au.`Audit Complete` = 1 THEN 1
        ELSE 0
      END `Audit Complete`,
      CASE
        WHEN ex.Exclude = 1 THEN 1
        ELSE 0
      END Exclusions
    FROM account_list__view ac
    LEFT JOIN role_map__df__view ro
      ON ac.`Owner: Full Name` = ro.`Account Owner/Sales Rep`
    LEFT JOIN audit_detail_mtd__view au
      ON ac.`Account 18 ID` = au.`Account 18 ID`
    LEFT JOIN exclusion__view ex
      ON ac.`Account 18 ID` = ex.`Account Id`
  ''')

In [0]:
# display(account_list_final__df.limit(10))
account_list_final__df.show(10, truncate=False)

+---------+-------------+------------------+------------------------------+------------------+------------------------------------------------------+---------+---------------------+-------------------------------+-------------------------+--------------------+------------+----------------+-------------------------+----------------------------+----------------------------------+-----------------------------+------------------------------+------+-------------------------------------+-------------+-----------------+------------+----------------------------------+------+-------------+------------------------------+-------------------+---------------------+--------------+----------+
Channel |Business Type|Customer No# (ERP)|Account Name |Account 18 ID |Street |City |Outlet Classification|PRS Group Outlet Type (EU) name|PRS Outlet Type (EU) name|PRS Image Level Name|Chain |Taxonomy Level 1|PRS Volume Potential Name|PRS Business Potential: Name|PRS Brand Building Potential: Name|Outlet Geolocation (Latitude)|Outlet Geolocation (Longitude)|Status|Visit Frequency: Visit Frequency Name|Preferred Day|Owner: Full Name |Country |Territory: Territory Name |Region|Company Type |Key Account Manager: Full Name|Area Sales Director|Account Owner Manager|Audit Complete|Exclusions|
+---------+-------------+------------------+------------------------------+------------------+------------------------------------------------------+---------+---------------------+-------------------------------+-------------------------+--------------------+------------+----------------+-------------------------+----------------------------+----------------------------------+-----------------------------+------------------------------+------+-------------------------------------+-------------+-----------------+------------+----------------------------------+------+-------------+------------------------------+-------------------+---------------------+--------------+----------+
OFF TRADE|Outlet |37202 |Game Liquors (Zevenwacht) 327|001Tn000003FFmpIAG|Shop 220, Zevenwacht Mall, Van Riebeeck Rd, Kuilsriver|Bellville|Retail Franchise |ZA RETAIL GROCERY |Group Corporate |null |Game Liquors|Game Liquors |null |High |High |-34,000,706,872,934,400 |18,726,913,360,896,200 |Active|2/Month W1 & 3 |Thursday |Curven Swiegelaar|South Africa|Cape Hybrid Inland (1) - Off Trade|South |Sales Service|null |Karen Wahl |Johan van Niekerk |1 |0 |
OFF TRADE|Outlet |37202 |Game Liquors (Zevenwacht) 327|001Tn000003FFmpIAG|Shop 220, Zevenwacht Mall, Van Riebeeck Rd, Kuilsriver|Bellville|Retail Franchise |ZA RETAIL GROCERY |Group Corporate |null |Game Liquors|Game Liquors |null |High |High |-34,000,706,872,934,400 |18,726,913,360,896,200 |Active|2/Month W1 & 3 |Thursday |Curven Swiegelaar|South Africa|Cape Hybrid Inland (1) - Off Trade|South |Sales Service|null |Karen Wahl |Johan van Niekerk |1 |0 |
OFF TRADE|Outlet |37202 |Game Liquors (Zevenwacht) 327|001Tn000003FFmpIAG|Shop 220, Zevenwacht Mall, Van Riebeeck Rd, Kuilsriver|Bellville|Retail Franchise |ZA RETAIL GROCERY |Group Corporate |null |Game Liquors|Game Liquors |null |High |High |-34,000,706,872,934,400 |18,726,913,360,896,200 |Active|2/Month W1 & 3 |Thursday |Curven Swiegelaar|South Africa|Cape Hybrid Inland (1) - Off Trade|South |Sales Service|null |Karen Wahl |Johan van Niekerk |1 |0 |
OFF TRADE|Outlet |37202 |Game Liquors (Zevenwacht) 327|001Tn000003FFmpIAG|Shop 220, Zevenwacht Mall, Van Riebeeck Rd, Kuilsriver|Bellville|Retail Franchise |ZA RETAIL GROCERY |Group Corporate |null |Game Liquors|Game Liquors |null |High |High |-34,000,706,872,934,400 |18,726,913,360,896,200 |Active|2/Month W1 & 3 |Thursday |Curven Swiegelaar|South Africa|Cape Hybrid Inland (1) - Off Trade|South |Sales Service|null |Karen Wahl |Johan van Niekerk |1 |0 |
OFF TRADE|Outlet |37202 |Game Liquors (Zevenwacht) 327|001Tn000003FFmpIAG|Shop 220, Zevenwacht Mall, Van Riebeeck Rd, Kuilsriver|Bellville|Retail Franchise |ZA RETAIL GROCERY |Group Corporate |null |Game

In [0]:
account_list_final__df.printSchema()

root
-- Channel: string (nullable = true)
-- Business Type: string (nullable = true)
-- Customer No# (ERP): string (nullable = true)
-- Account Name: string (nullable = true)
-- Account 18 ID: string (nullable = true)
-- Street: string (nullable = true)
-- City: string (nullable = true)
-- Outlet Classification: string (nullable = true)
-- PRS Group Outlet Type (EU) name: string (nullable = true)
-- PRS Outlet Type (EU) name: string (nullable = true)
-- PRS Image Level Name: string (nullable = true)
-- Chain: string (nullable = true)
-- Taxonomy Level 1: string (nullable = true)
-- PRS Volume Potential Name: string (nullable = true)
-- PRS Business Potential: Name: string (nullable = true)
-- PRS Brand Building Potential: Name: string (nullable = true)
-- Outlet Geolocation (Latitude): string (nullable = true)
-- Outlet Geolocation (Longitude): string (nullable = true)
-- Status: string (nullable = true)
-- Visit Frequency: Visit Frequency Name: string (nullable = true)
-- Preferred Day: string (nullable = true)
-- Owner: Full Name: string (nullable = true)
-- Country: string (nullable = true)
-- Territory: Territory Name: string (nullable = true)
-- Region: string (nullable = true)
-- Company Type: string (nullable = true)
-- Key Account Manager: Full Name: string (nullable = true)
-- Area Sales Director: string (nullable = true)
-- Account Owner Manager: string (nullable = true)
-- Audit Complete: integer (nullable = false)
-- Exclusions: integer (nullable = false)

#### `New tables` for export: "Audit gap list" series

In [0]:
account_list_final__df.createOrReplaceTempView("account_list_final__view")

`Previous` tab, from "audit gap list" excel
<br>
Pending:
<br>
- Does not refer to the model, confirm data source.
<br>
- Here we assume that "MTD Audit %" column is calculated as proposed below.
<br>
- COUNT or SUM, add DISTINCT?
<br>
- "Owner: Full Name" = "Account (EU): Owner Name" ?
<br>
Note: consider that in the template there are no formulas for these fields.

In [0]:
previous__df = spark.sql('''
  SELECT
    *,
    ROUND(`Sum of Audits Completed`/`Count of Account Name`*100, 0) AS `MTD Audit %`
  FROM (
    SELECT 
      `Area Sales Director` , -- regional director
      `Account Owner Manager`, -- sales manager
      `Owner: Full Name` AS `Account (EU): Owner Name`, -- sales rep
      CONCAT(`Account Owner Manager`,`Owner: Full Name`) AS concatenated_names,
      COUNT(`Account Name`) AS `Count of Account Name`,
      SUM(`Audit Complete`) AS `Sum of Audits Completed` 
    FROM account_list_final__view
    GROUP BY
      `Area Sales Director` ,
      `Account Owner Manager`,
      `Owner: Full Name`,
      CONCAT(`Account Owner Manager`,`Owner: Full Name`)
    ) t
  ''')

# display(previous__df)
previous__df.show(10, truncate=False)

+-------------------+---------------------+------------------------+-------------------------------+---------------------+-----------------------+-----------+
Area Sales Director|Account Owner Manager|Account (EU): Owner Name|concatenated_names |Count of Account Name|Sum of Audits Completed|MTD Audit %|
+-------------------+---------------------+------------------------+-------------------------------+---------------------+-----------------------+-----------+
Oscar Sithole |Rufus Moleko |Tshepo Masilela |Rufus MolekoTshepo Masilela |9163 |9152 |100.0 |
Oscar Sithole |Tshepo Mokale |Seoketso Kotane |Tshepo MokaleSeoketso Kotane |3773 |3718 |99.0 |
null |null |Tsediso Phantsi |null |111 |0 |0.0 |
Celeste Carstens |Brandon Pillay |Themba Myeni |Brandon PillayThemba Myeni |7484 |7483 |100.0 |
Karen Wahl |Ross Bennett |Simpiwe Dambuza |Ross BennettSimpiwe Dambuza |3169 |3124 |99.0 |
Oscar Sithole |Rufus Moleko |Brian Mvelase |Rufus MolekoBrian Mvelase |456 |360 |79.0 |
Celeste Carstens |Brandon Pillay |Vincent Msowoya |Brandon PillayVincent Msowoya |5073 |5051 |100.0 |
Wayne MacFarlane |Johnny Joubert |Lungile Itumeleng |Johnny JoubertLungile Itumeleng|6101 |6081 |100.0 |
Oscar Sithole |Rufus Moleko |Thabo Ubisi |Rufus MolekoThabo Ubisi |5161 |5127 |99.0 |
Celeste Carstens |Ray Saunders |Mlungisi Magwaxaza |Ray SaundersMlungisi Magwaxaza |7010 |6989 |100.0 |
+-------------------+---------------------+------------------------+-------------------------------+---------------------+-----------------------+-----------+
only showing top 10 rows

In [0]:
previous__df.createOrReplaceTempView("previous__view")

%md
`Pivot Audit Report` tab, from "audit gap list" excel.
<br>
Pending:
- "Previous # of Accounts" = "Count of Account 18 ID" due to the need to review the "previous" tab data source
<br>

In [0]:
pivot_audit_report__df = spark.sql('''

  SELECT 
   t.*,
   p.`Count of Account Name` AS `Previous # of Accounts`
  FROM (
    SELECT 
      `Area Sales Director` , -- regional director
      `Account Owner Manager`, -- sales manager
      `Owner: Full Name` AS `Account (EU): Owner Name`, -- sales rep
      CONCAT(`Account Owner Manager`, `Owner: Full Name`) AS concatenated_names,
      COUNT(`Account 18 ID`) AS `Count of Account 18 ID`,
      COUNT(DISTINCT `Account 18 ID`) AS `Distinct Count of Account 18 ID`
    FROM account_list_final__view
    GROUP BY
      `Area Sales Director` ,
      `Account Owner Manager`,
      `Owner: Full Name`,
      CONCAT(`Account Owner Manager`, `Owner: Full Name`)
    ) t
  LEFT JOIN previous__view p
    ON t.concatenated_names = p.concatenated_names
  ''')

# display(pivot_audit_report__df)
pivot_audit_report__df.show(10, truncate=False)

+-------------------+---------------------+------------------------+-------------------------------+----------------------+-------------------------------+----------------------+
Area Sales Director|Account Owner Manager|Account (EU): Owner Name|concatenated_names |Count of Account 18 ID|Distinct Count of Account 18 ID|Previous # of Accounts|
+-------------------+---------------------+------------------------+-------------------------------+----------------------+-------------------------------+----------------------+
Oscar Sithole |Rufus Moleko |Tshepo Masilela |Rufus MolekoTshepo Masilela |9163 |85 |9163 |
Oscar Sithole |Tshepo Mokale |Seoketso Kotane |Tshepo MokaleSeoketso Kotane |3773 |102 |3773 |
Celeste Carstens |Brandon Pillay |Themba Myeni |Brandon PillayThemba Myeni |7484 |86 |7484 |
Karen Wahl |Ross Bennett |Simpiwe Dambuza |Ross BennettSimpiwe Dambuza |3169 |91 |3169 |
Oscar Sithole |Rufus Moleko |Brian Mvelase |Rufus MolekoBrian Mvelase |456 |101 |456 |
Celeste Carstens |Brandon Pillay |Vincent Msowoya |Brandon PillayVincent Msowoya |5073 |88 |5073 |
Wayne MacFarlane |Johnny Joubert |Lungile Itumeleng |Johnny JoubertLungile Itumeleng|6101 |85 |6101 |
Oscar Sithole |Rufus Moleko |Thabo Ubisi |Rufus MolekoThabo Ubisi |5161 |106 |5161 |
Celeste Carstens |Ray Saunders |Mlungisi Magwaxaza |Ray SaundersMlungisi Magwaxaza |7010 |102 |7010 |
Celeste Carstens |Andrew Morris |Lebogang Maaga |Andrew MorrisLebogang Maaga |2811 |96 |2811 |
+-------------------+---------------------+------------------------+-------------------------------+----------------------+-------------------------------+----------------------+
only showing top 10 rows

%md
`Audit Gap List` tab, from "audit gap list" excel.
<br>
Pending:
- lorem ipusm
<br>

In [0]:
audit_gap_list__df = spark.sql('''
  SELECT
    `Account Name`,
    `Audit Complete`,
    Channel,
    `Account 18 ID`,
    `Area Sales Director`,
    `Account Owner Manager`,
    `Owner: Full Name`,
    `Region`,
    `Territory: Territory Name`,
    Chain,
    `Taxonomy Level 1`,
    `Visit Frequency: Visit Frequency Name`,
    `Key Account Manager: Full Name`,
    `Outlet Classification`,
    `PRS Group Outlet Type (EU) name`,
    `PRS Outlet Type (EU) name`,
    `PRS Image Level Name`,
    `PRS Volume Potential Name`,
    SUM(Exclusions) AS `Sum of Exclusions`
  FROM account_list_final__view
  GROUP BY
    `Account Name`,
    `Audit Complete`,
    Channel,
    `Account 18 ID`,
    `Area Sales Director`,
    `Account Owner Manager`,
    `Owner: Full Name`,
    `Region`,
    `Territory: Territory Name`,
    Chain,
    `Taxonomy Level 1`,
    `Visit Frequency: Visit Frequency Name`,
    `Key Account Manager: Full Name`,
    `Outlet Classification`,
    `PRS Group Outlet Type (EU) name`,
    `PRS Outlet Type (EU) name`,
    `PRS Image Level Name`,
    `PRS Volume Potential Name`  
  ''')

# display(audit_gap_list__df)
audit_gap_list__df.show(10, truncate=False)

+---------------------------------+--------------+---------+------------------+-------------------+---------------------+----------------+-------+--------------------------------------+-----------------+----------------------+-------------------------------------+------------------------------+---------------------+-------------------------------+-------------------------+--------------------+-------------------------+-----------------+
Account Name |Audit Complete|Channel |Account 18 ID |Area Sales Director|Account Owner Manager|Owner: Full Name|Region |Territory: Territory Name |Chain |Taxonomy Level 1 |Visit Frequency: Visit Frequency Name|Key Account Manager: Full Name|Outlet Classification|PRS Group Outlet Type (EU) name|PRS Outlet Type (EU) name|PRS Image Level Name|PRS Volume Potential Name|Sum of Exclusions|
+---------------------------------+--------------+---------+------------------+-------------------+---------------------+----------------+-------+--------------------------------------+-----------------+----------------------+-------------------------------------+------------------------------+---------------------+-------------------------------+-------------------------+--------------------+-------------------------+-----------------+
Tops 5 Star 21672 |1 |OFF TRADE|001Tn000003FEv5IAG|Oscar Sithole |Rufus Moleko |Tshepo Masilela |North |Limpopo Mpumalanga Hybrid (8) - Hybrid|Spar Group |Spar Inland |2/Month W1 & 3 |Jacques Weideman |Retail Grocery |ZA RETAIL GROCERY |Group Franchise |null |null |0 |
Oasis Maboneng |1 |OFF TRADE|001Tn000006hIJEIA2|Oscar Sithole |Tshepo Mokale |Seoketso Kotane |Central|Central Metro Hybrid 2 (3) - Off Trade|Independent |null |2/Month W1 & 4 |null |null |ZA COUNTER SERVE |Independent Independent |null |null |0 |
Loco Liquors (Brakendowns) |1 |OFF TRADE|001Tn000003FDALIA4|Oscar Sithole |Tshepo Mokale |Seoketso Kotane |Central|Central Metro Hybrid 2 (3) - Off Trade|Lifestyle Liquor |Lifestyle Liq - Retail|2/Month W2 & 4 |null |Retail Franchise |ZA RETAIL OTHER |Group Franchise |null |null |0 |
Makro (Crown Mines) |0 |OFF TRADE|001Tn000003FDUcIAO|Oscar Sithole |Tshepo Mokale |Seoketso Kotane |Central|Central Metro Hybrid 2 (3) - Off Trade|Makro |Makro |Weekly |Chad Haverly |Wholesaler |ZA WHOLESALER |Group Corporate |null |null |0 |
Marys Inn |0 |OFF TRADE|001Tn000003FDhXIAW|Oscar Sithole |Tshepo Mokale |Seoketso Kotane |Central|Central Metro Hybrid 2 (3) - Off Trade|Independent |Independent |Weekly |null |Independent |ZA COUNTER SERVE |Independent Independent |null |null |0 |
Loco Liquors (Mondeor) |0 |OFF TRADE|001Tn000003FEv0IAG|Oscar Sithole |Tshepo Mokale |Seoketso Kotane |Central|Central Metro Hybrid 2 (3) - Off Trade|Lifestyle Liquor |Lifestyle Liq - Retail|2/Month W1 & 3 |null |Retail Franchise |ZA RETAIL OTHER |Group Franchise |null |null |0 |
Liquor City (Bassonia) (Cheq) |0 |OFF TRADE|001Tn000003FGKPIA4|Oscar Sithole |Tshepo Mokale |Seoketso Kotane |Central|Central Metro Hybrid 2 (3) - Off Trade|Liquor City |Liquor City Corporate |Weekly |Daryl Pillay |Retail Franchise |ZA RETAIL OTHER |Group Corporate |null |null |0 |
PNP Family - Brcknhrst (GF04) |1 |OFF TRADE|001Tn000003FGWrIAO|Oscar Sithole |Tshepo Mokale |Seoketso Kotane |Central|Central Metro Hybrid 2 (3) - Off Trade|Pick n Pay |PNP Family Store |2/Month W2 & 4 |Jacques Weideman |Retail Grocery |ZA RETAIL GROCERY |Group Franchise |null |null |0 |
South Hills BBL (IGP127) (7 days)|0 |OFF TRADE|001Tn000003FGmTIAW|Oscar Sithole |Tshepo Mokale |Seoketso Kotane |Central|Central Metro Hybrid 2 (3) - Off Trade|Blue Bottle Group|Blue Bottle Group |2/Month W2 & 4 |Daryl Pillay |Retail Franchise |ZA RETAIL OTHER |Group Franchise |null |null |0 |
Tops Ridgeway(Gardens) 21489 |0 |OFF TRADE|001Tn000003FHS1IAO|Oscar Sithole |Tshepo Mokale |Seoketso Kotane |Central|Central Metro Hybrid 2 (3) - Off Trade|Spar Group |Spar Inland |2/Month W1 & 3 |Jacques Weideman |Retail Grocery |ZA RETAIL GROCERY |Group Franchise |null |null |0 |
+----

%md
`Audit Summary` tab, from "audit gap list" excel.
<br>
Pending:
- review "previous" table source
<br>

In [0]:
pivot_audit_report__df.createOrReplaceTempView("pivot_audit_report__view")

audit_summary__df = spark.sql('''
  SELECT 
    piv.`Area Sales Director` , -- regional director
    piv.`Account Owner Manager`, -- sales manager
    piv.`Account (EU): Owner Name`, -- sales rep
    piv.`Count of Account 18 ID` AS `Count of Account Name`,
    piv.`Distinct Count of Account 18 ID` AS `Sum of Audits Completed`,
    ROUND(
      COALESCE(
        piv.`Distinct Count of Account 18 ID` / NULLIF(piv.`Count of Account 18 ID`, 0)
        , 0)
        *100
      ,0) 
      AS `MTD Audit %`,
    pre.`MTD Audit %` AS `Prior Week`,
    pre.`Count of Account Name` AS `Previous # of Accounts`,
    piv.`Count of Account 18 ID`- pre.`Count of Account Name` AS `Database Movement`,
    piv.`Count of Account 18 ID`- piv.`Distinct Count of Account 18 ID` AS `No of Accounts Not Audited`
  FROM pivot_audit_report__view piv
  LEFT JOIN previous__view pre
    ON piv.concatenated_names = pre.concatenated_names
  ''')

# display(audit_summary__df)
audit_summary__df.show(10, truncate=False)


+-------------------+---------------------+------------------------+---------------------+-----------------------+-----------+----------+----------------------+-----------------+--------------------------+
Area Sales Director|Account Owner Manager|Account (EU): Owner Name|Count of Account Name|Sum of Audits Completed|MTD Audit %|Prior Week|Previous # of Accounts|Database Movement|No of Accounts Not Audited|
+-------------------+---------------------+------------------------+---------------------+-----------------------+-----------+----------+----------------------+-----------------+--------------------------+
Oscar Sithole |Rufus Moleko |Tshepo Masilela |9163 |85 |1.0 |100.0 |9163 |0 |9078 |
Oscar Sithole |Tshepo Mokale |Seoketso Kotane |3773 |102 |3.0 |99.0 |3773 |0 |3671 |
Celeste Carstens |Brandon Pillay |Themba Myeni |7484 |86 |1.0 |100.0 |7484 |0 |7398 |
Karen Wahl |Ross Bennett |Simpiwe Dambuza |3169 |91 |3.0 |99.0 |3169 |0 |3078 |
Oscar Sithole |Rufus Moleko |Brian Mvelase |456 |101 |22.0 |79.0 |456 |0 |355 |
Celeste Carstens |Brandon Pillay |Vincent Msowoya |5073 |88 |2.0 |100.0 |5073 |0 |4985 |
Wayne MacFarlane |Johnny Joubert |Lungile Itumeleng |6101 |85 |1.0 |100.0 |6101 |0 |6016 |
Oscar Sithole |Rufus Moleko |Thabo Ubisi |5161 |106 |2.0 |99.0 |5161 |0 |5055 |
Celeste Carstens |Ray Saunders |Mlungisi Magwaxaza |7010 |102 |1.0 |100.0 |7010 |0 |6908 |
Celeste Carstens |Andrew Morris |Lebogang Maaga |2811 |96 |3.0 |98.0 |2811 |0 |2715 |
+-------------------+---------------------+------------------------+---------------------+-----------------------+-----------+----------+----------------------+-----------------+--------------------------+
only showing top 10 rows

#### `Move` files btw Sharepoint Folders

##### keep researching -> move_to or moveto don't work in this module version --> Error: access denied

https://github.com/vgrem/Office365-REST-Python-Client/issues/157
<br>
https://github.com/vgrem/Office365-REST-Python-Client/issues/703

Before June 30th, 2023 The moveTo function was moving the files within the sharepoint folders. But after 1st July , 2023 , its not moving the files and giving the error as ('-2147024891, System.UnauthorizedAccessException', 'Access denied.', "403 Client Error: Forbidden for url:.....). I checked there are required access as It does other operations like delete and other. I guess the issue is with office365 python package.

#### `Write` in sharepoint

In [0]:
# Folder link and object
output_folder = "https://test-company.sharepoint.com/:f:/s/SASIPAutomationProject/Ehr1xSw61FRCnUk8aCvZ4NQB_qOJe7eqGjYdd0jXq9zrsg?e=MsHZxK"
output_folder_obj = client.web.get_folder_by_guest_url_extended(output_folder).execute_query()

#### Write in sharepoint - `A: Pandas`
##### simplier but less scalable if big data related

In [0]:
# Convert Spark DataFrame to Pandas DataFrame
role_map__pdf = role_map__df.toPandas()

# Convert the Pandas DataFrame to a CSV string
csv_data = role_map__pdf.to_csv(index=False)

# Upload CSV to SharePoint directly from the CSV string
output_file_name = "role_map.csv"
output_folder_obj.upload_file(output_file_name, csv_data.encode('utf-8')).execute_query()

Out[158]: /sites/SASIPAutomationProject/Shared Documents/General/SIP Automation Phase 2 Initial Documents/Phase 4 Top 10 reports/MTD Audit/Example of refreshing/POC - raw files/output/role_map.csv

#### Write in sharepoint - B: `just PySpark`
##### use A.. 

In [0]:
from io import StringIO

# Collect the DataFrame to a list of Row objects
rows = role_map__df.collect()

# Get the schema (column names)
columns = role_map__df.columns

# Create a CSV string from the collected data
csv_output = StringIO()
csv_output.write(','.join(columns) + '\n')  # Write header
for row in rows:
    csv_output.write(','.join(str(value) for value in row) + '\n')  # Write each row

# Get the CSV string from the StringIO object
csv_data = csv_output.getvalue()

# Upload CSV to SharePoint directly from the CSV string
output_file_name = "role_map.csv"
output_folder_obj.upload_file(output_file_name, csv_data.encode('utf-8')).execute_query()


Out[57]: /sites/SASIPAutomationProject/Shared Documents/General/SIP Automation Phase 2 Initial Documents/Phase 4 Top 10 reports/MTD Audit/Example of refreshing/POC - raw files/output/role_map.csv

#### `Delete` files in a Sharepoint Folder

In [0]:
# Folder link
deleteFolder_link = "https://test-company.sharepoint.com/:f:/s/SASIPAutomationProject/Ehr1xSw61FRCnUk8aCvZ4NQB_qOJe7eqGjYdd0jXq9zrsg?e=MsHZxK"

# Retrieve the folder object
deleteFolder_obj = client.web.get_folder_by_guest_url_extended(deleteFolder_link).execute_query()

# Load the files in the folder
sharepoint_files_toDelete = deleteFolder_obj.files
client.load(sharepoint_files_toDelete).execute_query()

# Delete each file in the folder
for file in sharepoint_files_toDelete:
    print(f"Deleting file: {file.properties['Name']}")
    file.delete_object()
    client.execute_query()

print("All files deleted.")

Deleting file: role_map.csv
All files deleted.